<a href="https://colab.research.google.com/github/thanhtruchhh/Data_Wrangling_Practice/blob/main/data_wrangling_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You have 2 sample dataset:
* **prices.csv**: Records of product's history price changes in Sep 2018.
* **sales.csv**: Records of product's sales in Sep 2018.

Use any data wrangling tools to calculate to total revenue of each product of each price. The solution must be applicable to similar datasets with large number of records.

# Data processing

In [1]:
from google.colab import files

In [2]:
uploaded = files.upload()

Saving prices.csv to prices (1).csv
Saving sales.csv to sales (1).csv


In [3]:
import pandas as pd
import numpy as np

In [4]:
sales = pd.read_csv('sales.csv')
sales.head()

,product_id,ordered_at,quantity_ordered
0,3998909,9/18/18 17:51,1
1,3998909,9/18/18 12:52,1
2,3998909,9/18/18 11:33,1
3,3998909,9/18/18 18:47,1
4,3998909,9/18/18 17:36,1


In [5]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        175 non-null    int64 
 1   ordered_at        175 non-null    object
 2   quantity_ordered  175 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ KB


The `ordered_at` column should have a datetime data type.

In [6]:
# Convert ordered_at to datetime
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
print(sales['ordered_at'].dtype)

datetime64[ns]


In [7]:
prices = pd.read_csv('prices.csv')
prices.head()

,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32


In [8]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  14 non-null     int64 
 1   old_price   14 non-null     int64 
 2   new_price   14 non-null     int64 
 3   updated_at  14 non-null     object
dtypes: int64(3), object(1)
memory usage: 576.0+ bytes


The `updated_at` column should have a datetime data type.

In [9]:
# Convert updated_at to datetime
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
print(prices['updated_at'].dtype)

datetime64[ns]


In [10]:
prices.describe()

,product_id,old_price,new_price
count,1.400000e+01,14.000000,14.000000
mean,3.737769e+06,66378.571429,62607.142857
std,1.077369e+06,61568.025900,53992.584452
min,6.400000e+01,15500.000000,15500.000000
25%,3.954203e+06,53500.000000,52375.000000
50%,3.998909e+06,60000.000000,57750.000000
75%,4.085861e+06,63625.000000,62000.000000
max,4.085861e+06,270000.000000,239000.000000


There are no unusual or unexpected values in the `old_price` and `new_price` columns.

# Data merging

Before using `merge_asof`, make sure that both DataFrames are sorted by the key.

In [11]:
# Sort the sales df
sorted_sales = sales.sort_values(by = 'ordered_at')
sorted_sales

,product_id,ordered_at,quantity_ordered
86,3954203,2018-09-11 01:43:00,1
28,4085861,2018-09-11 06:26:00,1
26,4085861,2018-09-11 06:53:00,1
27,4085861,2018-09-11 08:24:00,1
123,4085861,2018-09-11 09:30:00,1
...,...,...,...
67,4085861,2018-09-18 20:23:00,1
77,4085861,2018-09-18 20:43:00,1
79,4085861,2018-09-18 20:54:00,1
87,3954203,2018-09-18 21:26:00,1


## Nearest search

* Select the row in the **prices** DataFrame whose `updated_at` is closest to `ordered_at` in the **sales** DataFrame.
* Compare `ordered_at` and nearest `updated_at` of each order to find the correct price:
  * If `ordered_at` < `updated_at` &rarr; price = `old_price`.
  * Else price = `new_price`.

In [12]:
# Sort the prices df
sorted_prices = prices.sort_values(by = 'updated_at')
sorted_prices

,product_id,old_price,new_price,updated_at
4,3954203,68800,60000,2018-09-10 16:32:00
7,3998909,19000,17000,2018-09-10 16:35:00
0,64,270000,239000,2018-09-10 16:37:00
11,4085861,60000,53500,2018-09-11 08:51:00
1,3954203,60000,64000,2018-09-11 11:54:00
9,4085861,53500,67000,2018-09-12 03:51:00
6,3998909,17000,15500,2018-09-13 06:43:00
13,4085861,67000,62500,2018-09-13 06:43:00
3,3954203,64000,60500,2018-09-15 03:49:00
12,4085861,62500,58000,2018-09-15 03:51:00


In [13]:
# Merge nearest 2 df
sales_prices_merge_nr = pd.merge_asof(sorted_sales,
                             sorted_prices,
                             by = 'product_id',
                             left_on = 'ordered_at',
                             right_on = 'updated_at',
                             direction = 'nearest')

sales_prices_merge_nr

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00
...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500,52000,2018-09-17 22:59:00
171,4085861,2018-09-18 20:43:00,1,53500,52000,2018-09-17 22:59:00
172,4085861,2018-09-18 20:54:00,1,53500,52000,2018-09-17 22:59:00
173,3954203,2018-09-18 21:26:00,1,60500,57500,2018-09-17 22:59:00


In [14]:
# Add the final_price column
sales_prices_merge_nr['final_price'] = np.where(sales_prices_merge_nr['ordered_at'] >= sales_prices_merge_nr['updated_at'],
                                     sales_prices_merge_nr['new_price'],
                                     sales_prices_merge_nr['old_price'])

In [15]:
# Calc total line
sales_prices_merge_nr['revenue'] = sales_prices_merge_nr['quantity_ordered'] * sales_prices_merge_nr['final_price']

In [16]:
# Calc total revenue group by product, price
revenue_by_product_price = sales_prices_merge_nr.groupby(['product_id', 'final_price'])['revenue'].agg('sum').reset_index()
revenue_by_product_price

,product_id,final_price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,180000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,231000
6,3998909,17000,34000
7,4085861,52000,1040000
8,4085861,53500,2140000
9,4085861,58000,2204000


In [37]:
# Calculate total revenue by product
revenue_by_product = sales_prices_merge_nr.groupby('product_id')['revenue'].agg('sum').reset_index()
revenue_by_product

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0


## Backward and forward searh

* Backward search: Find the closest previous price change record based on `ordered_at` &rarr; Set price = `new_price`. In this case, we encounter `NULL` values in some rows as a product was ordered before the first time changing price.
* Forward search: Merge the above DataFrame with the **prices** DataFrame to find the closest next price change &rarr; Fill `NULL` values by `old_price`.

In [17]:
# Merge backward 2 df
sales_prices_merge_bw = pd.merge_asof(sorted_sales,
                                      sorted_prices,
                                      by = 'product_id',
                                      left_on = 'ordered_at',
                                      right_on = 'updated_at')

sales_prices_merge_bw

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,68800.0,60000.0,2018-09-10 16:32:00
1,4085861,2018-09-11 06:26:00,1,NaN,NaN,NaT
2,4085861,2018-09-11 06:53:00,1,NaN,NaN,NaT
3,4085861,2018-09-11 08:24:00,1,NaN,NaN,NaT
4,4085861,2018-09-11 09:30:00,1,60000.0,53500.0,2018-09-11 08:51:00
...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500.0,52000.0,2018-09-17 22:59:00
171,4085861,2018-09-18 20:43:00,1,53500.0,52000.0,2018-09-17 22:59:00
172,4085861,2018-09-18 20:54:00,1,53500.0,52000.0,2018-09-17 22:59:00
173,3954203,2018-09-18 21:26:00,1,60500.0,57500.0,2018-09-17 22:59:00


In [18]:
# Set price = new_price
sales_prices_merge_bw['final_price'] = sales_prices_merge_bw['new_price']

In [19]:
# Merge sales_prices_merge_fw with price with direction = forward
sales_prices_merge_fw = pd.merge_asof(sales_prices_merge_bw,
                                      sorted_prices,
                                      by = 'product_id',
                                      left_on = 'ordered_at',
                                      right_on = 'updated_at',
                                      direction = 'forward')

sales_prices_merge_fw

,product_id,ordered_at,quantity_ordered,old_price_x,new_price_x,updated_at_x,final_price,old_price_y,new_price_y,updated_at_y
0,3954203,2018-09-11 01:43:00,1,68800.0,60000.0,2018-09-10 16:32:00,60000.0,60000.0,64000.0,2018-09-11 11:54:00
1,4085861,2018-09-11 06:26:00,1,NaN,NaN,NaT,NaN,60000.0,53500.0,2018-09-11 08:51:00
2,4085861,2018-09-11 06:53:00,1,NaN,NaN,NaT,NaN,60000.0,53500.0,2018-09-11 08:51:00
3,4085861,2018-09-11 08:24:00,1,NaN,NaN,NaT,NaN,60000.0,53500.0,2018-09-11 08:51:00
4,4085861,2018-09-11 09:30:00,1,60000.0,53500.0,2018-09-11 08:51:00,53500.0,53500.0,67000.0,2018-09-12 03:51:00
...,...,...,...,...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500.0,52000.0,2018-09-17 22:59:00,52000.0,NaN,NaN,NaT
171,4085861,2018-09-18 20:43:00,1,53500.0,52000.0,2018-09-17 22:59:00,52000.0,NaN,NaN,NaT
172,4085861,2018-09-18 20:54:00,1,53500.0,52000.0,2018-09-17 22:59:00,52000.0,NaN,NaN,NaT
173,3954203,2018-09-18 21:26:00,1,60500.0,57500.0,2018-09-17 22:59:00,57500.0,NaN,NaN,NaT


In [20]:
# Fill na
sales_prices_merge_fw['final_price'] = sales_prices_merge_fw['final_price'].fillna(sales_prices_merge_fw['old_price_y'])

# Keep important fields
sales_prices_merge_fw = sales_prices_merge_fw[['product_id', 'ordered_at', 'quantity_ordered', 'final_price']]
sales_prices_merge_fw.head()

,product_id,ordered_at,quantity_ordered,final_price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,60000.0
2,4085861,2018-09-11 06:53:00,1,60000.0
3,4085861,2018-09-11 08:24:00,1,60000.0
4,4085861,2018-09-11 09:30:00,1,53500.0


In [21]:
# Calc total line
sales_prices_merge_fw['revenue'] = sales_prices_merge_fw['quantity_ordered'] * sales_prices_merge_fw['final_price']

# Calc total revenue group by product, price
revenue_by_product_price_bf = sales_prices_merge_fw.groupby(['product_id', 'final_price'])['revenue'].agg('sum').reset_index()
revenue_by_product_price_bf

<ipython-input-21-e7bcf3c9df22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_prices_merge_fw['revenue'] = sales_prices_merge_fw['quantity_ordered'] * sales_prices_merge_fw['final_price']


,product_id,final_price,revenue
0,64,239000.0,956000.0
1,3954203,57500.0,57500.0
2,3954203,60000.0,180000.0
3,3954203,64000.0,640000.0
4,3998909,15500.0,15500.0
5,3998909,16500.0,231000.0
6,3998909,17000.0,34000.0
7,4085861,52000.0,1040000.0
8,4085861,53500.0,2140000.0
9,4085861,58000.0,2204000.0


In [22]:
# Calc total revenue by product
revenue_by_product_bf = sales_prices_merge_fw.groupby('product_id')['revenue'].agg('sum').reset_index()
revenue_by_product_bf

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0
